In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.optimize as sopt
import sklearn.datasets
import random

#Beta Methods

In [2]:
#Fletcher-reeves conjugate gradient update
def beta_fr(gradient, old_gradient):
  beta = (gradient@gradient.T)/(old_gradient@old_gradient.T)
  return beta

#Polak-Riberie conjugate gradient update
def beta_pr(gradient, old_gradient):
  y_hat = gradient - old_gradient
  beta = (gradient@y_hat.T)/(old_gradient@old_gradient.T)
  return beta

#Line Search Methods

In [6]:
#Exact Line Search (Bisection) method
def bisection(func, x, direction, eps=1e-3):
  a = 0
  b = 1
  c = (b+a)/2
  error = abs(b-a)
  count = 0
  def f(t):
    value, grad = func(x + (t*direction), 1)
    return abs(grad)

  while error > eps:
    if f(a) < f(b):
      b = c 
      error = abs(b-a)
    else:
      a = c
      error = abs(b-a)
    c_old = c
    c = (b+a)/2
    dif = abs(c - c_old)
    if dif < eps:
      return c
    count += 1
  return c

In [7]:
#Backtracking line search
def backtracking_cg(func, x, direction, iterations = 10, eps = 1e-5 ):
    alpha = 0.25
    beta = 0.5
    t = 1
    count = 0
    n = len(direction)
    approx = -float('inf')
    while func(x + t*direction, order = 0) >= approx and count <= iterations:
        val,grad = func(x, order = 1)
        approx = val + alpha*t*grad*direction.T
        t = t*beta
        count += 1
    return t 

#Conjugate Gradient Algorithm

In [3]:
def cg( func, initial_x, beta_method = beta_fr, eps=1e-5, maximum_iterations=65536, linesearch=None, *linesearch_args  ):
#def cg( func, initial_x, beta_method = beta_fr, eps=1e-5, maximum_iterations=65536, linesearch=None, *linesearch_args  ):
    """ 
    Conjugate Gradient
    func:               the function to optimize It is called as "value, gradient = func( x, 1 )
    initial_x:          the starting point
    eps:                the maximum allowed error in the resulting stepsize t
    maximum_iterations: the maximum allowed number of iterations
    linesearch:         the linesearch routine
    *linesearch_args:   the extra arguments of linesearch routine
    """

    if eps <= 0:
        raise ValueError("Epsilon must be positive")
    x = np.asarray( initial_x.copy() )

    # initialization
    values = []
    runtimes = []
    xs = []
    start_time = time.time()
    m = len( x )
    iterations = 0
    direction = np.asmatrix( np.zeros( x.shape ) )

    # conjugate gradient updates
    while True:
        value, gradient = func( x , 1 )
        value = np.double( value )
        gradient = np.asarray( gradient )
        
        # updating the logs
        values.append( value )
        runtimes.append( time.time() - start_time )
        xs.append( x.copy() )

        # termination criteria
        if np.linalg.norm(gradient) < eps:
          break
        # updating beta and direction
        if iterations == 0 or iterations % m == 0:
          beta = 0
          direction = -1*gradient
        else:
          beta = beta_method(gradient, old_gradient)
          direction = -1*gradient + (beta*direction)

        # updating t
        if linesearch is None:
          t = 1
        else:
          t = linesearch(func, x, direction, *linesearch_args)
        # updating x, gradient and iteration
        #x += t * direction
        x = x + (t*direction)
        old_gradient = gradient
        iterations += 1
        if iterations >= maximum_iterations:
            return (x, values, runtimes, xs)
            raise ValueError("Too many iterations")

    return (x, values, runtimes, xs)

Function to evaluate is PSF function:
$$ \min_{-10 \leq x_i \leq 10} (x_1+10x_2)^2+5(x_3-x_4)^2+(x_2-2x_3)^4 + 10(x_1-x_4)^4,$$

In [8]:
def psf_func(x, order=0):
  x = np.asmatrix(x).T
  part1 = (x[0] + 10*x[1])**2
  part2 = 5*(x[2]-x[3])**2
  part3 = (x[1] - 2*x[2])**4
  part4 = 10*(x[0]-x[3])**4
  der1 = 2*(20*(x[0]-x[2])**3 + x[0] + 10*x[1])
  der2 = 4*(5*(x[0]+10*x[1]) + (x[1]-2*x[2])**3)
  der3 = 10*(x[2]-x[3] - 8*(x[1]-2*x[2])**3)
  der4 = 10*(-4*(x[0]-x[3])**3 - x[2] + x[3])
  value = part1 + part2 + part3 + part4
  gradient = [[]]
  if order == 0:
    return value
  elif order == 1:
    gradient = np.asmatrix([der1.item(), der2.item(), der3.item(), der4.item()])
    return (value, gradient)
  else:
    raise ValueError("The argument \"order\" should be 0 or 1")

In [9]:
#Testing XPOWELL function
betas = [beta_fr, beta_pr]
for method in betas:
    x, values, runtimes, xs = cg(psf_func, [0.5,0.5,0.5,0.5], beta_method=method, linesearch=backtracking_cg)
    if method is beta_fr:
      print(f'With Fletcher-Reeves gradient update: Iterations= {len(xs)}, solution= {x}')
    else:
      print(f'With Polak-Riberie gradient update: Iterations= {len(xs)}, solution= {x}')

With Fletcher-Reeves gradient update: Iterations= 9033, solution= [[ 0.00849477 -0.00084948  0.00238687  0.00238791]]
With Polak-Riberie gradient update: Iterations= 13132, solution= [[ 0.00848065 -0.00084808  0.00238285  0.00238401]]
